In [0]:
import os
import numpy as np
import pandas as pd
import gc

In [0]:
#2. Get the file
if os.name=='posix':
    
    # GOOGLE COLAB SETUP
    # Load the Drive helper and mount
    from google.colab import drive
    # This will prompt for authorization.
    drive.mount('/content/drive')
    
    data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'

### TOXIC WIKIPEDIA

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [41]:
!wget http://www.bannedwordlist.com/lists/swearWords.csv

--2020-05-31 17:27:16--  http://www.bannedwordlist.com/lists/swearWords.csv
Resolving www.bannedwordlist.com (www.bannedwordlist.com)... 52.218.84.188
Connecting to www.bannedwordlist.com (www.bannedwordlist.com)|52.218.84.188|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480 [application/octet-stream]
Saving to: ‘swearWords.csv.1’

swearWords.csv.1    100%[===================>]     480  --.-KB/s    in 0s      

2020-05-31 17:27:16 (79.7 MB/s) - ‘swearWords.csv.1’ saved [480/480]



In [0]:
sw_words = pd.read_csv("swearWords.csv").columns

In [0]:
def toxicity(text, dict_toxic):
  example_score = 0
  for word in text['text'].split():
    if word in dict_toxic:
      example_score += 1
    else:
      continue
  return example_score

In [0]:
from tqdm import tqdm 
tqdm.pandas()

In [45]:
test_preds = pd.DataFrame(X_test, columns=['text']).progress_apply(lambda x: toxicity(x, sw_words), axis=1)

100%|██████████| 64830/64830 [00:02<00:00, 23132.98it/s]


In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(pd.DataFrame(test_preds, columns=['preds']))
test_preds_scaled = scaler.transform(pd.DataFrame(test_preds, columns=['preds']))

In [0]:
from sklearn.metrics import roc_auc_score
from tensorflow.keras import backend as K
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score

In [48]:
for thr in [0.001]:
  threshold = thr
  probs_class = test_preds_scaled.copy()
  probs_class[probs_class >= threshold] = 1 
  probs_class[probs_class < threshold] = 0
  auc_f   = average_precision_score(y_test, test_preds_scaled)
  roc_f   = roc_auc_score(y_test, test_preds_scaled)
  f_score = f1_score(y_test, probs_class)

  print(f'{threshold} av precision {auc_f}  auc_roc {roc_f}   f_score {f_score}')

0.001 av precision 0.3749956409693343  auc_roc 0.7180562545507059   f_score 0.5316366827555946


### Facebook

In [0]:
#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/facebook_cleaned_no_punkt.csv')

train.drop(['id', 'agr'], axis=1, inplace=True)
train.text.fillna("empty", inplace=True)

rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [50]:
test_preds = pd.DataFrame(X_test, columns=['text']).progress_apply(lambda x: toxicity(x, sw_words), axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(pd.DataFrame(test_preds, columns=['preds']))
test_preds_scaled = scaler.transform(pd.DataFrame(test_preds, columns=['preds']))

100%|██████████| 4616/4616 [00:00<00:00, 33367.47it/s]


In [58]:
for thr in [0.001]:
  threshold = thr
  probs_class = test_preds_scaled.copy()
  probs_class[probs_class >= threshold] = 1 
  probs_class[probs_class < threshold] = 0
  auc_f   = average_precision_score(y_test, test_preds_scaled)
  roc_f   = roc_auc_score(y_test, test_preds_scaled)
  f_score = f1_score(y_test, probs_class)

  print(f'{threshold} av precision {auc_f}  auc_roc {roc_f}   f_score {f_score}')

0.001 av precision 0.5732144094937575  auc_roc 0.5103974074770944   f_score 0.05832410483573274


### Twitter

In [0]:

#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/twitter_cleaned_no_punkt.csv')

In [0]:
train.drop(['count', 'hate_speech', 'offensive_language', 'neither', 'class'], axis=1, inplace=True)
train.tweet.fillna("empty", inplace=True)
train = train.drop_duplicates(subset=['tweet', 'mal'])

In [0]:
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.tweet, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )

In [100]:
test_preds = pd.DataFrame(X_test).rename(columns={'tweet':'text'}).progress_apply(lambda x: toxicity(x, sw_words), axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(pd.DataFrame(test_preds, columns=['preds']))
test_preds_scaled = scaler.transform(pd.DataFrame(test_preds, columns=['preds']))




  0%|          | 0/7168 [00:00<?, ?it/s]


100%|██████████| 7168/7168 [00:00<00:00, 42307.33it/s]


In [101]:
for thr in [0.1]:
  threshold = thr
  probs_class = test_preds_scaled.copy()
  probs_class[probs_class >= threshold] = 1 
  probs_class[probs_class < threshold] = 0
  auc_f   = average_precision_score(y_test, test_preds_scaled)
  roc_f   = roc_auc_score(y_test, test_preds_scaled)
  f_score = f1_score(y_test, probs_class)

  print(f'{threshold} av precision {auc_f}  auc_roc {roc_f}   f_score {f_score}')

0.1 av precision 0.9342917004834034  auc_roc 0.8053381754984442   f_score 0.7800324675324675


### Formspring

In [0]:
train         = pd.read_csv(f'{data_path}/formspring_cleaned_no_punkt.csv') 

In [0]:
train.text.fillna("empty", inplace=True)

In [0]:
rs=42
from sklearn.model_selection import train_test_split      
X, X_test, y, y_test  = train_test_split(train.text, train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [87]:
test_preds = pd.DataFrame(X_test, columns=['text']).progress_apply(lambda x: toxicity(x, sw_words), axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(pd.DataFrame(test_preds, columns=['preds']))
test_preds_scaled = scaler.transform(pd.DataFrame(test_preds, columns=['preds']))




  0%|          | 0/3817 [00:00<?, ?it/s]


100%|██████████| 3817/3817 [00:00<00:00, 32666.24it/s]


In [96]:
for thr in [0.1]:
  threshold = thr
  probs_class = test_preds_scaled.copy()
  probs_class[probs_class >= threshold] = 1 
  probs_class[probs_class < threshold] = 0
  auc_f   = average_precision_score(y_test, test_preds_scaled)
  roc_f   = roc_auc_score(y_test, test_preds_scaled)
  f_score = f1_score(y_test, probs_class)

  print(f'{threshold} av precision {auc_f}  auc_roc {roc_f}   f_score {f_score}')

0.1 av precision 0.20300700759687193  auc_roc 0.7046294335808103   f_score 0.30806257521058966
